# Советы по выбору столбцов в DataFrame

<a href="https://t.me/init_python"><img src="https://dfedorov.spb.ru/pandas/logo-telegram.png" width="35" height="35" alt="telegram" align="left"></a>

<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/быстрое%20введение%20в%20pandas/Советы%20по%20выбору%20столбцов%20в%20DataFrame.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory" target="_blank"></a>

## Введение

В этом Блокноте мы обсудим несколько советов по использованию [`iloc`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html) для работы с набором данных, содержащим большое количество столбцов. Даже если у вас есть некоторый опыт использования [`iloc`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html), следует изучить пару полезных приемов, чтобы ускорить анализ и избежать ввода большого количества имен столбцов в коде.

> Оригинал статьи Криса [тут](https://pbpython.com/selecting-columns.html)

## Почему мы заботимся о выборе столбцов?

Во многих стандартных примерах, встречающихся в науке о данных, относительно небольшое число столбцов. Например, в наборе данных `Titanic` их 8, у `Iris` - 4, а у `Boston Housing` - 14. Реальные же наборы данных - грязные и часто включают множество дополнительных (потенциально ненужных) столбцов.

В процессе анализа данных вам может потребоваться выбрать подмножество столбцов по следующим причинам:

- Фильтрация для включения отдельных столбцов позволяет уменьшить объем памяти и ускорить обработку данных.
- Ограничение количества столбцов может уменьшить накладные расходы, связанные с хранением модели данных в вашей голове (см. [Магическое число семь плюс-минус два](https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D0%B3%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D1%87%D0%B8%D1%81%D0%BB%D0%BE_%D1%81%D0%B5%D0%BC%D1%8C_%D0%BF%D0%BB%D1%8E%D1%81-%D0%BC%D0%B8%D0%BD%D1%83%D1%81_%D0%B4%D0%B2%D0%B0)).
- При изучении нового набора данных может потребоваться разбить задачу на управляемые части.
- В некоторых случаях может потребоваться перебрать столбцы и выполнить вычисления или очистку, чтобы получить данные в формате, необходимом для дальнейшего анализа.
- Ваши данные могут содержать лишнюю или повторяющуюся информацию.

Описанные ниже приемы помогут сократить время, которое вы тратите на обработку столбцов данных.

## Данные

Чтобы проиллюстрировать некоторые примеры, я собираюсь использовать необычный [набор данных](https://data.cityofnewyork.us/Environment/2018-Central-Park-Squirrel-Census-Squirrel-Data/vfnx-vebw) из [переписи белок Центрального парка](https://www.thesquirrelcensus.com/). Да, видимо, в Центральном парке пытались подсчитать и занести в каталог белок. Я подумал, что это будет забавный пример для работы. 

Этот набор данных включает 3023 строки данных и 31 столбец. Хотя 31 столбец не является огромным количеством столбцов, это полезный пример для иллюстрации концепций, которые вы можете применить к данным с большим количеством столбцов.

> *Прим. переводчика*: на сайте Центрального парка содержится [подробная инструкция](https://data.cityofnewyork.us/api/views/vfnx-vebw/files/038f2dd2-2eb6-4152-968a-b075705c9986?download=true&filename=User%20Guide%20_%20Central%20Park%20Squirrel%20Census%20Data%20Collection.docx) по работе с данными. Разберем ее подробно:

В октябре 2018 года с помощью добровольцев-охотников за белками подсчитали количество белок в Центральном парке Нью-Йорка. В результате переписи белок был выпущен отчет. Параметры, включенные в отчет:

- `X`: координата долготы точки наблюдения за белкой
- `Y`: Координата широты точки наблюдения за белкой
- `Unique Squirrel ID`: идентификационный ярлык для каждой обнаруженной белки. Тег состоит из `Hectare ID` + `Shift` + `Date` (MMDD) + `Hectare Squirrel Number`.
- `Hectare`: ID тег, полученный из сетки гектаров, используемой для разделения и подсчета парковой зоны. Одна ось, которая проходит преимущественно с севера на юг, является числовой (1-42), а ось, которая проходит преимущественно с востока на запад, является алфавитной (A-I).
- `Shift`: значение - `AM` или `PM`, чтобы указать, когда произошло наблюдение - утром или поздно вечером.
- `Date`: объединение месяца, дня и года наблюдения (MMDDYYYY).
- `Hectare Squirrel Number`: число в хронологической последовательности наблюдений за белками для отдельного наблюдения.
- `Age`: значение `Adult` (Взрослый) or `Juvenile` (Несовершеннолетний).
- `Primary Fur Color`: `Gray`, `Cinnamon` или `Black`.
- `Highlight Fur Color`: дискретное значение или строковые значения, состоящие из `Gray`, `Cinnamon`, `Black` или `White`.
- `Combination of Primary and Highlight Color`: комбинация двух предыдущих столбцов; в этом столбце приведены общие наблюдаемые перестановки основных цветов и оттенков.
- `Color Notes`: иногда наблюдатели добавляли комментарии о состоянии беличьего меха. 
- `Location`: `Ground Plane` или `Above Ground`. Наблюдателям было дано указание отметить, где была белка, когда ее впервые заметили.
- `Above Ground Sighter Measurement`: `FALSE` - для наблюдений за белками на плоскости земли.
- `Specific Location`: Иногда наблюдатели добавляли комментарии о местонахождении белки.
- `Running`: была замечена бегущая белка.
- `Chasing`: белка, преследующая другую белку.
- `Climbing`: белка, взбирающаяся на дерево или другой природный объект.
- `Eating`: белка за едой.
- `Foraging`: белка в поисках пищи.
- `OtherActivities`: другая активность белки. 
- `Kuks`: веселое голосовое общение, используемое белками по разным причинам.
- `Quaas`: удлиненное голосовое общение, которое может указывать на присутствие наземного хищника, такого как собака.
- `Moans`: высокий голос, который может указывать на присутствие воздушного хищника, такого как ястреб.
- `Tail Flags`: белка, ловящая хвост. Используется для увеличения размера белки и сбивания с толку соперников или хищников. 
- `Tail Twitches`: используется белкой для выражения интереса, любопытства.
- `Approaches`: белка, приближающаяся к человеку в поисках еды.
- `Indifferent`: белке было безразлично присутствие человека.
- `Runs From`: белка убегает от людей, считая их угрозой.
- `Other Interactions`: наблюдатель отмечает другие типы взаимодействий между белками и людьми.

Уверен, теперь вы узнали много нового о поведении белок! 

Давайте начнем с чтения данных:

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# прямая ссылка на данные: 'https://data.cityofnewyork.us/api/views/vfnx-vebw/rows.csv?accessType=DOWNLOAD&bom=true&format=true'

# скачал набор на случай изменений в исходном:
df = pd.read_csv('https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/%D0%B1%D1%8B%D1%81%D1%82%D1%80%D0%BE%D0%B5%20%D0%B2%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%B2%20pandas/data/2018_Central_Park_Squirrel_Census_-_Squirrel_Data.csv')

In [ ]:
df.head()

Иногда бывает сложно запомнить имена всех столбцов и их индекс. 

Вот простое решение: 

In [ ]:
col_mapping = [f"{c[0]}:{c[1]}" for c in enumerate(df.columns)]

Получился такой список:

In [ ]:
col_mapping

## Использование iloc

Основная функция, которую мы рассмотрим, - это [`iloc`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html). 

Она используется для индексации на основе целых чисел. Поскольку функции `iloc` и `loc` могут принимать в качестве входных данных логический массив, бывают случаи, когда эти функции производят одинаковый вывод. Однако в рамках этого Блокнота я сосредоточусь только на выборе столбца с помощью `iloc`.

Вот простой рисунок, иллюстрирующий основное использование `iloc`:

![](https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/pic/iloc.png)

Например, если вы хотите посмотреть столбец данных `Unique Squirrel ID` для всех строк:

In [ ]:
df.iloc[:, 2]

Посмотреть в дополнение к `Unique Squirrel ID` местоположение `X` и `Y` :

In [ ]:
df.iloc[:, [0, 1, 2]]

Ввод всех столбцов не самый эффективный способ, поэтому можем использовать нотацию срезов:

In [ ]:
df.iloc[:, 0:3] # df.iloc[:, :3]

Это даст тот же результат, что и выше.

Если хочется объединить список целых чисел с нотацией среза? 

Можно попробовать что-то вроде такого:

In [ ]:
# произойдет ошибка: invalid syntax

#df.iloc[:, [0:3, 15:19]]

или такого:

In [ ]:
# произойдет ошибка: Too many indexers

#df.iloc[:, 0:3,15:19]

Хммм... очевидно, это не работает.

К счастью, есть объект NumPy [`r_`](https://numpy.org/doc/stable/reference/generated/numpy.r_.html), который может нам помочь. 

Объект `r_` "преобразует объекты срезов в конкатенацию по первой оси". 

Вот немного более сложный пример, демонстрирующий, как это работает:

In [ ]:
np.r_[0:3, 15:19, 24, 25]

Это круто! 

Объект `r_` преобразовал комбинацию целочисленных списков и нотации срезов в единый список, который мы можем передать `iloc`:

In [ ]:
df.iloc[:, np.r_[0:3, 15:19, 24, 25]]

Вот еще один совет: вы можете использовать эту нотацию при чтении данных с помощью `read_csv`:

In [ ]:
df_2 = pd.read_csv(
    'https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/%D0%B1%D1%8B%D1%81%D1%82%D1%80%D0%BE%D0%B5%20%D0%B2%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%B2%20pandas/data/2018_Central_Park_Squirrel_Census_-_Squirrel_Data.csv',
    usecols=np.r_[1, 2, 5:8, 15:25]
)

In [ ]:
df_2.head()

Я считаю эту нотацию полезной, когда есть набор данных, в котором вы хотите оставить столбцы и не хотите вводить их полные имена.

> Нужно быть осторожным при использовании нотации среза и помнить, что последнее число в диапазоне не включается в сгенерированный список чисел.

Например, если мы укажем диапазон `2:4`, мы получим только список из `2` и `3`:

In [ ]:
np.r_[2:4]

Если вы хотите включить индекс столбца `4`, используйте `np.r_[2:5]`.

У `np.r_` есть необязательный аргумент `step`. 

В следующем примере можем указать, что список будет увеличиваться на 2:

In [ ]:
np.r_[2:10:2]

## iloc и логические массивы

Один из наиболее эффективных способов фильтрации столбцов - передать в `iloc` логический массив. 

Самая важная идея заключается в том, что мы не создаем логический массив вручную, а используем вывод другой функции pandas для генерации массива и передачи его в `iloc`.

В данном случае можем использовать метод доступа `str` для индекса столбца, как и любой другой столбец данных pandas. Это сгенерирует необходимый логический массив, который ожидает `iloc`. 

Например, хотим увидеть, название каких столбцов содержит слово `run`:

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html

run_cols = df.columns.str.contains('run', case=False) # не чувствительный к регистру
run_cols

Передадим новый массив логических значений в `iloc`, чтобы выбрать два столбца:

In [ ]:
df.iloc[:, run_cols].head()

На практике чаще используют лямбда-функцию:

In [ ]:
df.iloc[:, lambda df:df.columns.str.contains('run', 
                                             case=False)].head()

Преимущество в использовании функций `str` заключаются в том, что вы можете усложнить работу с потенциальными параметрами фильтрации. 

Например, если мы хотим, чтобы все столбцы содержали в названии `Color` или `Tail`:

In [ ]:
df.iloc[:, lambda df: df.columns.str.contains('Color|Tail',
                                              case=False)].head()

Мы можем объединить все эти концепции вместе, используя результаты логического массива для получения индекса, а затем использовать `np.r_` для объединения списков.

> Пример ниже можно упростить, используя `filter`. 

Вот пример, в котором мы хотим получить все столбцы, связанные с `Color` или `Tail`, а также `Unique Squirrel ID` белки:

In [ ]:
color_cols = df.columns.str.contains('Color|Tail', case=False)
color_cols

In [ ]:
color_indices = [i for i, col in enumerate(color_cols) if col]
color_indices

In [ ]:
df.iloc[:, np.r_[0:3, color_indices]].head()

## Фильтр

В исходном Блокноте я не включил никакой информации об использовании [`filter`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.filter.html) для выбора столбцов. `filter` звучит так, будто его следует использовать для фильтрации данных, а не имен столбцов. К счастью, в pandas вы можете использовать `filter` для выбора столбцов!

Если вы хотите выбрать столбцы, в названии которых встречается `Color`, то можете использовать следующий код:

In [ ]:
df.filter(like='Color')

Вы можете использовать регулярное выражение, чтобы найти столбцы, содержащие один или несколько шаблонов:

In [ ]:
df.filter(regex='ing|Date')

Пример, показанный выше, можно более лаконично записать с помощью `filter`:

In [ ]:
df.filter(regex='Color|Tail')

> Предостережение: имейте в виду, что при изменении порядка следования столбцов могут возникнуть сложности при обработке данных показанным выше способом.